**NOTE**: Here we test the robustness of the STged with  Gene expression deconvolution methods on the mouse FISH+ datasets


In [1]:
#remove(list = ls())
#define the conda env

library(reticulate)
use_condaenv("ccnet", required = TRUE)


In [2]:
library(ggplot2)
library(scatterpie)
library(RColorBrewer)

In [3]:
# Source code
source("./code/STged.R")
source("./code/benchmark.R")

In [4]:
colors <- colorRampPalette(brewer.pal(9, "Set3"))(12)


# Load data the generated simulated data
##  we use the FISH+ data from the mouse tissue. There are 69 spots.

In [5]:
sim_data = readRDS("./realdata/FISH+/sim_fishplus_input.RDS")

TRUE_F = sim_data$trueF
beta.type = sim_data$beta.type

cell_type = colnames(beta.type)
cell_type

## cell type proportion predected from the proposed cell type deconvolution methods
res_decon_ct = readRDS("./realdata/FISH+/EnDecon_major/FISH_Results.Deconv.RDS")


[1] "astrocytes" "endo_mural" "eNeuron"    "iNeuron"    "microglia" 
[6] "Olig"

# Run the STged step by step

## Step 1: clear data

In [6]:
depthscale = 1e6
clean.only = FALSE
datax = data_process(sc_exp = sim_data$sc_exp,   sc_label = sim_data$sc_label, 
                     spot_exp = sim_data$spot_exp,  spot_loc = sim_data$spot_loc,
                     depthscale = depthscale,  gene_det_in_min_cells_per = 0.05, 
                     expression_threshold = 0,
                     nUMI = 100, verbose = FALSE, clean.only = clean.only)

for(i in 1:length(cell_type)){
  TRUE_F[[i]] = TRUE_F[[i]][,colnames(datax$spot_exp )]
  
}


##  true cell type-specific gene expression data

In [7]:
gene_sec = rownames(datax$sc_exp)
for(i in 1:length(cell_type)){
  TRUE_F[[i]] = TRUE_F[[i]][gene_sec,colnames(datax$spot_exp )]
  
}
# normaliztion of the true cell type-specific gene expression data
trueF = res_library_size_normaization(estres = TRUE_F, depthscale = depthscale )

# Evaluate the results: spot-level analysis

## Step 2: construct spatial correlation structures

## Step 3: construct reference gene matrix

In [8]:
cat("Construct spatial correlation", "\n")
L.mat = dis_weight(spot_loc = datax$spot_loc, spot_exp = datax$spot_exp, k = 6, 
                   quantile_prob_bandwidth = 1/3, method = "Hex", 
                   coord_type = "grid")

Construct spatial correlation 


In [9]:

cat("Construct reference gene matrix", "\n")

ref_exp = create_group_exp(sc_exp = datax$sc_exp, sc_label = datax$sc_label)

ref_exp = ref_exp[rownames(datax$spot_exp), cell_type]
colnames(ref_exp )

beta.type = beta.type[colnames(datax$spot_exp),]


Construct reference gene matrix 


[1] "astrocytes" "endo_mural" "eNeuron"    "iNeuron"    "microglia" 
[6] "Olig"

In [10]:
p = nrow(datax$spot_exp)
p 
K = ncol(beta.type)
beta.ind.temp = beta.type>0.05
beta.ind =  matrix(list(), K ,1)
for (i in 1:K){
  
  beta.ind[[i]] =  matrix( rep(beta.ind.temp[,i],p), nrow = p, byrow = TRUE)
}


[1] 6471

## Step 4: run the main model
### Step 4-1: run the main model with true cell type proportion

In [11]:
  start_time <- Sys.time()
  stged.est.true = MUR.STged(srt_exp = datax$spot_exp, ref_exp = ref_exp, 
                        beta.type = beta.type,   W = L.mat$dis_weight, 
                        lambda1 = NULL, lambda2 = NULL, cutoff = 0.05, 
                        epsilon = 1e-5)

  end_time <- Sys.time()

 

We will adpote a value for lambda 1 in our algorithm... 
Select value of lambda1 0.09479641 
tuning for lambda 2 in our algorithm... 
Select value of lambda2 0.1279606 
Run the main algorithm... 


# Run the STged with cell type proportion predicted from proposed cell type deconvolution methods

In [12]:
methods = names(res_decon_ct)
bench_celltype_STged = matrix(list(), length(methods) ,1)

for(ii in 1: length(methods) ){
  
  cat("Run the STged", methods[ii], "\n")
  beta.type.bench = res_decon_ct[[ii]]
  start_time <- Sys.time()
  stged.est = MUR.STged(srt_exp = datax$spot_exp, ref_exp = ref_exp, 
                        beta.type = beta.type.bench,   W = L.mat$dis_weight, 
                        lambda1 = NULL, lambda2 = NULL, cutoff = 0.05, 
                        epsilon = 1e-5)
                    
  end_time <- Sys.time()
  bench_celltype_STged[[ii]] = stged.est$V.hat
  cat("Run time of STged", end_time - start_time,"\n") 
  
}

bench_celltype_STged$trueF = trueF
bench_celltype_STged$ beta.ind = beta.ind

files = paste0("./results/FISH+/STged_est_spot_decon_tuning.RDS")

saveRDS(bench_celltype_STged, file =files)

Run the STged CARD 
We will adpote a value for lambda 1 in our algorithm... 
Select value of lambda1 0.1396419 
tuning for lambda 2 in our algorithm... 
Select value of lambda2 0.1279606 
Run the main algorithm... 
Run time of STged 4.704297 
Run the STged RCTD 
We will adpote a value for lambda 1 in our algorithm... 
Select value of lambda1 0.4088076 
tuning for lambda 2 in our algorithm... 
Select value of lambda2 0.1279606 
Run the main algorithm... 
Run time of STged 11.87745 
Run the STged SpatialDWLS 
We will adpote a value for lambda 1 in our algorithm... 
Select value of lambda1 0.09674089 
tuning for lambda 2 in our algorithm... 
Select value of lambda2 0.1279606 
Run the main algorithm... 
Run time of STged 3.52834 
Run the STged SPOTlight 
We will adpote a value for lambda 1 in our algorithm... 
Select value of lambda1 1.239609 
tuning for lambda 2 in our algorithm... 
Select value of lambda2 0.1279606 
Run the main algorithm... 
Run time of STged 16.37564 
Run the STged Ste